<a href="https://colab.research.google.com/github/michaelknorr/CHE4061/blob/main/CHE4061_Group_Tutorial_1_(Thermo_Analysis)_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Solve for an unknown duty value via an overall column enthalpy balance

Next cells define an example from GT 1

In [ ]:
from typing import List, Optional

def Distillation_Column_Energy_Balance(
    q_in: List[Optional[float]],
    h_in: List[Optional[float]],
    q_out: List[Optional[float]],
    h_out: List[Optional[float]],
    Q_reb: Optional[float] = None,
    Q_cond: Optional[float] = None,
    Q_lost: Optional[float] = None
) -> float:
    """
    Performs an energy balance for a distillation column.

    Parameters
    ----------
    q_in : List[Optional[float]]
        Mass flow rates of feed streams (unknown values should be None).
    h_in : List[Optional[float]]
        Enthalpy values of feed streams (unknown values should be None).
    q_out : List[Optional[float]]
        Mass flow rates of product streams (unknown values should be None).
    h_out : List[Optional[float]]
        Enthalpy values of product streams (unknown values should be None).
    Q_reb : Optional[float], default=None
        Reboiler duty (if known).
    Q_cond : Optional[float], default=None
        Condenser duty (if known).
    Q_lost : Optional[float], default=None
        Heat lost to the environment (if known).

    Returns
    -------
    float
        The energy imbalance value (should be 0 if balanced).
    """

    # Calculate total enthalpy in and out (ignore None values)
    H_in = sum(q * h for q, h in zip(q_in, h_in) if q is not None and h is not None)
    H_out = sum(q * h for q, h in zip(q_out, h_out) if q is not None and h is not None)

    # Set default values for unknown heat duties
    Q_reb = Q_reb if Q_reb is not None else 0
    Q_cond = Q_cond if Q_cond is not None else 0
    Q_lost = Q_lost if Q_lost is not None else 0

    # Energy balance equation: H_in + Q_reb - Q_cond - Q_lost = H_out
    energy_balance = H_in + Q_reb - Q_cond - Q_lost - H_out

    # Display results
    print("\n--- Energy Balance Summary ---")
    print(f"Total Enthalpy In: {H_in}")
    print(f"Total Enthalpy Out: {H_out}")
    print(f"Reboiler Duty (Q_reb): {Q_reb}" if Q_reb else "Q_reb is unknown")
    print(f"Condenser Duty (Q_cond): {Q_cond}" if Q_cond else "Q_cond is unknown")
    print(f"Heat Lost to Environment (Q_lost): {Q_lost}" if Q_lost else "Q_lost is unknown")

    if energy_balance == 0:
        print("\nEnergy balance is satisfied.")
    else:
        print(f"\nEnergy imbalance detected: {energy_balance} (Check input values or solve for unknowns)")

    return energy_balance  # Returning imbalance value




In [ ]:
qin1=445.5/2.205
qout1=175.5/2.205
qout2=270/2.205
qin1, qout1, qout2

(202.0408163265306, 79.59183673469387, 122.44897959183673)

In [ ]:
Distillation_Column_Energy_Balance([202.04], [17e3], [79.59, 122.44], [13420, 15840], None, 273e5, 0)


--- Energy Balance Summary ---
Total Enthalpy In: 3434680.0
Total Enthalpy Out: 3007547.4
Q_reb is unknown
Condenser Duty (Q_cond): 27300000.0
Q_lost is unknown

Energy imbalance detected: -26872867.4 (Check input values or solve for unknowns)


-26872867.4

**Irreversible Entropy Production Calculation and Example from GT 1b**

In [ ]:
from typing import List, Optional

def dS_irr(
    q_in: List[Optional[float]],
    s_in: List[Optional[float]],
    q_out: List[Optional[float]],
    s_out: List[Optional[float]],
    Q_reb: float,
    Q_cond: float,
    T_reb_steam: float,
    T_cond_cw: float
) -> float:
    """
    Calculates the irreversible entropy generation (dS_irr) for a separation process.

    Parameters
    ----------
    q_in : List[Optional[float]]
        Mass flow rates of inlet streams.
    s_in : List[Optional[float]]
        Entropy values of inlet streams.
    q_out : List[Optional[float]]
        Mass flow rates of outlet streams.
    s_out : List[Optional[float]]
        Entropy values of outlet streams.
    Q_reb : float
        Heat added to the reboiler (Joules).
    Q_cond : float
        Heat removed by the condenser (Joules).
    T_reb_steam : float
        Temperature of steam in the reboiler (Kelvin).
    T_cond_cw : float
        Temperature of cooling water in the condenser (Kelvin).

    Returns
    -------
    float
        Irreversible entropy generation (dS_irr) in J/K.
    """

    # Compute entropy contribution from streams
    S_in = sum(q * s for q, s in zip(q_in, s_in))
    S_out = sum(q * s for q, s in zip(q_out, s_out))

    # Compute heat transfer contributions
    dS_Q_reb = Q_reb / T_reb_steam
    dS_Q_cond = Q_cond / T_cond_cw

    # Compute entropy generation
    dS_irr = S_out + dS_Q_cond - S_in - dS_Q_reb

    return dS_irr


In [ ]:
dS_irr([202.04], [25.05], [79.59, 122.45], [5.87, 21.22], 26872867.4, 273e5, 373.15, 298.15)

17552.86901769633

**Lost Work Calculation**

In [ ]:
from typing import List, Optional, Callable

def lost_work(
    T0: float,
    dS_irr_function: Callable[..., float],
    *args, **kwargs
) -> float:
    """Computes lost work as T0 * dS_irr."""
    return T0 * dS_irr_function(*args, **kwargs)

In [ ]:
lost_work(
    298.15,
    dS_irr,
    q_in=[202.04],
    s_in=[25.05],
    q_out=[79.59, 122.45],
    s_out=[5.87, 21.22],
    Q_reb=26872867.4,
    Q_cond=273e5,
    T_reb_steam=373.15,
    T_cond_cw=298.15
)

5233387.89762616

**Minimum Work of Separation (Full Analysis)**

In [ ]:
from typing import List, Optional

def distillation_analysis(
    q_in: List[Optional[float]],
    s_in: List[Optional[float]],
    q_out: List[Optional[float]],
    s_out: List[Optional[float]],
    Q_reb: float,
    Q_cond: float,
    T_reb_steam: float,
    T_cond_cw: float,
    T0: float
):
    """
    Performs entropy generation, lost work, and minimum work calculations
    for a distillation column.

    Parameters:
    - q_in: List of mass flow rates for feed streams (kg/s)
    - s_in: List of specific entropies for feed streams (J/kg·K)
    - q_out: List of mass flow rates for product streams (kg/s)
    - s_out: List of specific entropies for product streams (J/kg·K)
    - Q_reb: Heat input to the reboiler (J)
    - Q_cond: Heat removed in the condenser (J)
    - T_reb_steam: Steam temperature in the reboiler (K)
    - T_cond_cw: Cooling water temperature in the condenser (K)
    - T0: Ambient temperature (K)

    Returns:
    - Prints values of entropy generation (S_irr), lost work (LW), and minimum work (W_min)
    """

    # Step 1: Calculate entropy generation (S_irr)
    S_in = sum(q * s for q, s in zip(q_in, s_in))
    S_out = sum(q * s for q, s in zip(q_out, s_out))
    dS_Q_reb = Q_reb / T_reb_steam
    dS_Q_cond = Q_cond / T_cond_cw
    S_irr = S_out + dS_Q_cond - S_in - dS_Q_reb

    # Step 2: Compute lost work (LW)
    LW = T0 * S_irr

    # Step 3: Compute minimum work (W_min)
    W_min = Q_reb * (1 - T0 / T_reb_steam) - Q_cond * (1 - T0 / T_cond_cw) - LW

    # Print results
    print("\n--- Distillation Column Analysis ---")
    print(f"Entropy Generation (S_irr): {S_irr:.2f} J/K")
    print(f"Lost Work (LW): {LW:.2f} J")
    print(f"Minimum Work (W_min): {W_min:.2f} J")

    return W_min  # Returning for potential further use

# Example Calculation
T0 = 298.15  # Surrounding temperature (K)
Q_reb = 26872867.4  # Joules
Q_cond = 273e5  # Joules
T_reb_steam = 373.15  # Kelvin
T_cond_cw = 298.15  # Kelvin

# Call function
distillation_analysis(
    q_in=[202.04],
    s_in=[25.05],
    q_out=[79.59, 122.45],
    s_out=[5.87, 21.22],
    Q_reb=Q_reb,
    Q_cond=Q_cond,
    T_reb_steam=T_reb_steam,
    T_cond_cw=T_cond_cw,
    T0=T0
)



--- Distillation Column Analysis ---
Entropy Generation (S_irr): 17552.87 J/K
Lost Work (LW): 5233387.90 J
Minimum Work (W_min): 167831.60 J


167831.59855499957

**Full Analysis with Efficiency calculation and Example!!**

In [ ]:
from typing import List, Optional

def distillation_analysis(
    q_in: List[Optional[float]],
    s_in: List[Optional[float]],
    q_out: List[Optional[float]],
    s_out: List[Optional[float]],
    Q_reb: float,
    Q_cond: float,
    T_reb_steam: float,
    T_cond_cw: float,
    T0: float
):
    """
    Performs entropy generation, lost work, minimum work, and thermodynamic efficiency
    calculations for a distillation column.

    Parameters:
    - q_in: List of mass flow rates for feed streams (kg/s)
    - s_in: List of specific entropies for feed streams (J/kg·K)
    - q_out: List of mass flow rates for product streams (kg/s)
    - s_out: List of specific entropies for product streams (J/kg·K)
    - Q_reb: Heat input to the reboiler (J)
    - Q_cond: Heat removed in the condenser (J)
    - T_reb_steam: Steam temperature in the reboiler (K)
    - T_cond_cw: Cooling water temperature in the condenser (K)
    - T0: Ambient temperature (K)

    Returns:
    - Prints values of entropy generation (S_irr), lost work (LW), minimum work (W_min), and thermodynamic efficiency
    """

    # Step 1: Calculate entropy generation (S_irr)
    S_in = sum(q * s for q, s in zip(q_in, s_in))
    S_out = sum(q * s for q, s in zip(q_out, s_out))
    dS_Q_reb = Q_reb / T_reb_steam
    dS_Q_cond = Q_cond / T_cond_cw
    S_irr = S_out + dS_Q_cond - S_in - dS_Q_reb

    # Step 2: Compute lost work (LW)
    LW = T0 * S_irr

    # Step 3: Compute minimum work (W_min)
    W_min = Q_reb * (1 - T0 / T_reb_steam) - Q_cond * (1 - T0 / T_cond_cw) - LW

    # Step 4: Calculate thermodynamic efficiency as W_min / (W_min + LW)
    if W_min + LW != 0:  # Avoid division by zero
        efficiency = W_min / (W_min + LW)
    else:
        efficiency = 0  # If W_min + LW is zero, efficiency is undefined

    # Print results
    print("\n--- Distillation Column Analysis ---")
    print(f"Entropy Generation (S_irr): {S_irr:.2f} J/K")
    print(f"Lost Work (LW): {LW:.2f} J")
    print(f"Minimum Work (W_min): {W_min:.2f} J")
    print(f"Thermodynamic Efficiency: {efficiency:.4f}")

    return W_min, efficiency  # Returning for potential further use

# Example Calculation
T0 = 298.15  # Surrounding temperature (K)
Q_reb = 26872867.4  # Joules
Q_cond = 273e5  # Joules
T_reb_steam = 373.15  # Kelvin
T_cond_cw = 298.15  # Kelvin

# Call function
distillation_analysis(
    q_in=[202.04],
    s_in=[25.05],
    q_out=[79.59, 122.45],
    s_out=[5.87, 21.22],
    Q_reb=Q_reb,
    Q_cond=Q_cond,
    T_reb_steam=T_reb_steam,
    T_cond_cw=T_cond_cw,
    T0=T0
)



--- Distillation Column Analysis ---
Entropy Generation (S_irr): 17552.87 J/K
Lost Work (LW): 5233387.90 J
Minimum Work (W_min): 167831.60 J
Thermodynamic Efficiency: 0.0311


(167831.59855499957, 0.03107290838183156)